In [ ]:
!pip install h2o

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#kaggle https://www.kaggle.com/c/santander-customer-transaction-prediction

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics




# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Checking to see the expected submission format
sample = pd.read_csv('../input/sample_submission.csv')

print(sample.head())

train = pd.read_csv('../input/train.csv')
print ("Train dataset has {} rows(samples) with {} columns(features) each.".format(*train.shape))
print(train.head())

#test = pd.read_csv('test.csv', index_col='Id')
test = pd.read_csv('../input/test.csv')

print ("Test dataset has {} rows(samples) with {} columns(features) each.".format(*test.shape))
print(test.head())

In [ ]:
'''#start H2O 
import h2o
from h2o.automl import H2OAutoML

#load data as h2o frames
h2o.init()
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

#drop passengerId from data set
Id = test['ID_code']
train = train.drop('ID_code',axis =1)
test = test.drop('ID_code',axis =1)

#identify predictors and labels
x = train.columns
y = 'target'
x.remove(y)

#for binary classification, lables should be a factor
train[y] = train[y].asfactor()

# Run AutoML
aml_ti = H2OAutoML(max_runtime_secs= 7200,max_models= 20, seed= 7,nfolds= 10)
aml_ti.train(x = x, y = y,
          training_frame = train)
          
#check the leaderboard
lb_ti = aml_ti.leaderboard
lb_ti

'''

In [ ]:
#prediction
'''lb_ti.head()
pred = aml_ti.leader.predict(test)
'''

In [ ]:
#start H2O 
import h2o
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

#load data as h2o frames
h2o.init()
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

#drop passengerId from data set
Id = test['ID_code']
train = train.drop('ID_code',axis =1)
test = test.drop('ID_code',axis =1)

#identify predictors and labels
x = train.columns
y = 'target'
x.remove(y)

train_split,validation_split = train.split_frame(ratios=[.7])

model_dl = H2ODeepLearningEstimator(hidden = [10, 10], epochs = 5)
model_dl.train(x, y, training_frame=train_split, validation_frame=validation_split)

pred = model_dl.predict(model_dl, newdata = test)
#save predict results to submission form
pred_df = pred.as_data_frame()
#pred_df['predict'].astype(int).head()
pred_res = pred_df['predict'].astype(int)
Id_df = Id.as_data_frame()
res_ti = pd.concat([Id_df,pred_res],axis=1,ignore_index = True)
res_ti.columns = ['ID_code','target']
res_ti.to_csv('subs.csv',index=False)

In [ ]:
res_ti.head()